# The JuMP ecosystem for mathematical optimization: MOI

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

In [11]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

LoadError: [91mArgumentError: Module MathOptInterfaceGLPK not found in current path.
Run `Pkg.add("MathOptInterfaceGLPK")` to install the MathOptInterfaceGLPK package.[39m

## MOI and JuMP

In [64]:
model = Model()
@variable(model, x[1:2])
@constraint(model, linear1, 2*x[1] +   x[2] <= 1)  # save reference linear1 for this constraint
@constraint(model, 2*x[1] + 2*x[2] <= 1) 
@constraint(model, x[1]^2 + x[2]^2 <= 1)  
@constraint(model, [1;x] in SecondOrderCone())  # ||x|| <= 1
@objective(model, Max, x[1] + x[2]) 

Behind every JuMP model there is a MOI model (used to be a MPB model)

In [6]:
typeof(model.moibackend)

MathOptInterface.Bridges.LazyBridgeOptimizer{MathOptInterface.Utilities.CachingOptimizer{MathOptInterface.AbstractOptimizer,MathOptInterface.Utilities.UniversalFallback{JuMP.JuMPMOIModel{Float64}}},MathOptInterface.Bridges.AllBridgedConstraints{Float64}}

### Recovering variables:

In [33]:
xx = MOI.get(model.moibackend,MOI.ListOfVariableIndices()) 

2-element Array{MathOptInterface.VariableIndex,1}:
 MathOptInterface.VariableIndex(1)
 MathOptInterface.VariableIndex(2)

In [39]:
x[1].index == xx[1]

true

In [42]:
x[1].m == model

true

In [84]:
typeof(x[1])

JuMP.VariableRef

JuMP variable : MOI variable + owning model
```julia
struct VariableRef 
    m::Model
    index::MOI.VariableIndex
end
```

MOI variable : Type-safe unique and not necessarily consecutive index
```julia
struct VariableIndex
    value::Int64
end
```

### Recovering constraints:

In [45]:
L,Q,C = MOI.get(model.moibackend,MOI.ListOfConstraints())

3-element Array{Tuple{DataType,DataType},1}:
 (MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64})   
 (MathOptInterface.ScalarQuadraticFunction{Float64}, MathOptInterface.LessThan{Float64})
 (MathOptInterface.VectorAffineFunction{Float64}, MathOptInterface.SecondOrderCone)     

MOI constraints are of the form 
$$
f(x) \in S
$$
were
* $f$ is an ``MOI.AbstractFunction`` (e.g. ``MOI.ScalarAffineFunction`` = $a^T x +b$ for some $a,b$) and
* $S$ is an ``MOI.AbstractSet`` (e.g. ``MOI.LessThan`` = $\{y\,:\, y\leq u\}$ for some $u$)

#### Linear inequalities (``MOI.ScalarAffineFunction``-in-``MOI.LessThan``):

In [53]:
linear = MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarAffineFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{L...}())

2-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(1)
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(2)

In [67]:
linear1.index == linear[1]

true

In [68]:
fieldnames(linear1)

2-element Array{Symbol,1}:
 :m    
 :index

In [69]:
linear1.m == model

true

In [85]:
typeof(linear1)

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}

JuMP constraint : MOI constraint + owning model
```julia
struct ConstraintRef{C}
    m::Model
    index::C             # C = ConstraintIndex{F,S} for some F,S
end
```

MOI constraint : Type-safe wrapper of index to constraint of class F-in-S. Index not necessarily consecutive and unique within constraint class
```julia
struct ConstraintIndex{F,S}
    value::Int64
end
```

Linear inequalities 1 : 
* ``2*x[1] + x[2] <= 1``, or
* $a^T x + b \in \{y:y\leq u\}$ for $a=(2,1)^T$, $b=0$ and $u=0$ 

$a=(2,1)^T$:

In [56]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).terms

2-element Array{MathOptInterface.ScalarAffineTerm{Float64},1}:
 MathOptInterface.ScalarAffineTerm{Float64}(2.0, MathOptInterface.VariableIndex(1))
 MathOptInterface.ScalarAffineTerm{Float64}(1.0, MathOptInterface.VariableIndex(2))

$b=0$:

In [58]:
MOI.get(model.moibackend,MOI.ConstraintFunction(),linear[1]).constant

0.0

$u=0$:

In [60]:
MOI.get(model.moibackend,MOI.ConstraintSet(),linear[1]).upper

1.0

Similar for othet constrsints:

In [49]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.ScalarQuadraticFunction{Float64}, MOI.LessThan{Float64}}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{Q...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.LessThan{Float64}}(3)

In [51]:
MOI.get(model.moibackend,MOI.ListOfConstraintIndices{MOI.VectorAffineFunction{Float64}, MOI.SecondOrderCone}())  #MOI.get(model.moibackend,MOI.ListOfConstraintIndices{C...}())

1-element Array{MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone},1}:
 MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(4)

## MOI Constraints

| Mathematical Constraint       | MOI Function                 | MOI Set        |
|-------------------------------|------------------------------|----------------|
| $a^Tx = b$                  | `ScalarAffineFunction`       | `EqualTo`      |
| $l \le a^Tx \le u$          | `ScalarAffineFunction`       | `Interval`     |
| $x_i \le u$                 | `SingleVariable`             | `LessThan`     |
| $Ax + b \in \mathbb{R}_+^n$ | `VectorAffineFunction`       | `Nonnegatives` |
| $\lVert Ax + b\rVert_2 \le c^Tx + d$                        | `VectorAffineFunction`       | `SecondOrderCone` |
| $y \exp (x/y) \le z, y > 0$  | `VectorOfVariables`       | `ExponentialCone`                  |
| $x \in \mathbb{R}^{d\left(d+1\right)/2}$,◹$(x)\in PSD \subseteq \mathbb{R}^{d\times d}$                                       | `VectorOfVariables`          | `PositiveSemidefiniteConeTriangle` |

## Bridges

Missing bridges https://github.com/JuliaOpt/MathOptInterface.jl/issues/423

unsupported constraints https://github.com/JuliaOpt/JuMP.jl/pull/1386